In [1]:
import numpy as np
import tensorflow as tf

# dynamic_rnn 의 sequence_length 옵션
# 가변 길이 단어를 학습시킬 수도 있습니다.
# 짧은 단어는 긴 단어와 차이만큼을 0 으로 채우고 길이를 계산해 sequence_length 넘겨주면 됩니다. (batch_size 배열로)
# 코드가 길어지므로 이해를 위해 이번 예시에서는 단어의 길이를 고정합니다.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

# one-hot
# {'a':0, 'b':1, 'c':2, 'd':3, 'e':4,  요렇게..   }

num_dic = {n: i for i, n in enumerate(char_arr)}

# enumerate는 "열거하다"라는 뜻
# 이 함수는 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력으로 받아
# 인덱스 값을 포함하는 enumerate 객체를 리턴한다.
# (출처: https://wikidocs.net/32)
temp_input_data = ['there', 'is', 'no', 'cow', 'level']
for num_word in enumerate(temp_input_data):
    print(num_word)

print('\n', num_dic)

dic_len = len(num_dic)

(0, 'there')
(1, 'is')
(2, 'no')
(3, 'cow')
(4, 'level')

 {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [3]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

# 입력 데이터는 sequential 이고, 앞에 세 글자를 입력(X), 뒤에 한 글자를 출력(Y)으로 사용할 것입니다.
# 예) 'love' 라는 단어에 대하여, X는 lov, Y는 e 가 됩니다.3

단어를 학습에 사용할 수 있는 형태로 변환 <br>
1. 입력값, 출력값의 알파벳 인덱스를 구함 <br>
2. 입력값을 원핫인코딩으로 변환 <br>

In [4]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    
    for seq in seq_data:
        input = [ num_dic[n] for n in seq[:-1] ] # 'wor'  # [22, 14, 17] 
        target = num_dic[seq[-1]] # 'd' # 3
        
        input_batch.append(np.eye(dic_len)[input])
        # eye(): ones on the diagonal and zeros elsewhere.
        # In : np.eye(3)
        # Out:
        # array([[1., 0., 0.],
        #        [0., 1., 0.],
        #        [0., 0., 1.]])
        
        # if input is [0, 1, 2]:
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        
        target_batch.append(np.eye(dic_len)[target])

    return input_batch, output_batch    

신경망 구성

In [5]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 10 # 30

n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input])
print(X)
Y = tf.placeholder(tf.float32, [None])
print(Y) # Y는 원핫이 아니라 그냥 인덱스 숫자를 이용함

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Tensor("Placeholder:0", shape=(?, 3, 26), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=float32)


RNN 셀 생성 <br>
Dropout 적용

In [6]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

RNN 신경망 생성

In [7]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) #순환 신경망
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

최종 출력값 ( RNN -> mnist 형식 )

In [8]:
outputs = tf.transpose(outputs, [1, 0, 2]) # perm = [1,0,2] ?
print(outputs)
outputs = outputs[-1]
print(outputs)
model = tf.matmul(outputs, W) + b
# 행렬 모양 너무 헷갈린다.. -_-.. 시각적으로 인지하려고 하지말고

Tensor("transpose:0", shape=(3, ?, 128), dtype=float32)
Tensor("strided_slice:0", shape=(?, 128), dtype=float32)
